In [1]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import pandas as pd
import numpy as np
import requests
import urllib
import re
from bs4 import BeautifulSoup
import bs4

In [3]:
df_news = pd.read_csv('data/Macquarie_19900101-20220420.csv')

In [ ]:
df_news

In [ ]:
#Uncomment to export the original dataframe into the excel format
#df_news.to_excel("output.xlsx")

In [ ]:
#Set as :20 to make a small sample
df_news = df_news.iloc[:]

In [ ]:
pd.set_option('display.max_colwidth', None)
df_news['URL'].astype(str)
df_news['URL'].fillna('nan')

In [4]:
# Loop through URLs, scrap information, identify the main body
# URL이 주어졌을 경우 해당 매체 웹사이트로부터 뉴스기사전문 크롤링

#최종결과물에 대한 텍스트 전처리-불용어 제거
#반복적인 불용어에 대한 메뉴얼 수정 내용 추가 - 미디어 매채별 예외처리 등록
def clean_html(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
#    cleanr = re.compile('[^-가-힣a-zA-Z0-9/ ]')
    clean_text = re.sub(cleanr, '', raw_html)
    clean_text = clean_text.replace('window.jQuery || document.write("")', '')
    clean_text = clean_text.replace('Copyright ⓒ 동아일보 &amp; donga.com', '') #동아일보
    clean_text = clean_text.replace('좋아요 이미지좋아요슬퍼요 이미지슬퍼요화나요 이미지화나요후속기사 원해요 이미지후속기사 원해요기사추천 이미지기사추천공유 이미지공유공유하기닫기', '')
    clean_text = clean_text.replace('ⓒ 매일경제 &amp; mk.co.kr', '').replace('googletag.display("google_dfp_MC_250x250")', '').replace('googletag.display("google_dfp_MC_2x1,fluid")', '')
    clean_text = clean_text.replace('[', '').replace(']', '').replace('&amp', '').replace(', ■', '').replace('■ &lt', '').replace(';', '').replace('◆', '')
    return clean_text


def crawling_main_text(url):
        
        if type(url)==float:
            content = 'n/a'
        
        
        #조선일보 - 미완성
        elif 'chosun' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")
            content_obj = soup.find_all("div", class_="article-body")
            
            
            content = clean_html(str(content_obj))
        
        
        #중앙일보
        elif 'joins' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_body")
            content = clean_html(str(content_obj))
        
        
        #동아일보
        elif 'donga' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_txt")
            content = clean_html(str(content_obj))
        
        
        #한겨례신문
        elif 'hani' in url:
            url = urllib.request.Request(url)
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
            
            content_obj = soup.find_all('div', class_="text")
            content = clean_html(str(content_obj))
        

        #경향신문
        elif 'khan' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('p', class_="content_text")
            content = clean_html(str(content_obj))
        
        
        #한국경제
        elif 'hankyung' in url:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
            url = urllib.request.Request(url, headers=headers)
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
    
            content_obj = soup.find_all('div', id="articletxt")
            content = clean_html(str(content_obj))
        
        
        #매일경제
        elif 'mk' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
            soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

            content_obj = soup_alt.find_all('div', class_="art_txt")

            content = clean_html(str(content_obj))
    
    
        return content
                                     

In [ ]:
#For Loop을 통한 본문 크롤링
#데이터 프레임에 추가

df_news['main_content']='n/a'

for i, url in enumerate(df_news['URL']):
    print(url)
    print(type(url))
    try:
        main_body = crawling_main_text(url)
        print(main_body)
        df_news['main_content'].iloc[i] = main_body
    except ConnectionError:
        continue
    

In [ ]:
#Export dataframe if necessary
df_news.to_excel("df_news_updated.xlsx")

  
  
  
  
  
## Reference Code for Individual Media Source  (Legacy; No Longer Updated)

### 한겨례

In [ ]:
import urllib.request
from bs4 import BeautifulSoup

def contents_hankyoreh():
    
    #샘플 기사 URL
    list_url = "https://www.hani.co.kr/arti/politics/politics_general/1038853.html"
    
    url = urllib.request.Request(list_url)
    res = urllib.request.urlopen(url).read().decode("utf-8")
    #print(res)
    
    soup = BeautifulSoup(res, "html.parser")
    params1 = []
    params2 = []
    
    for link1, link2 in zip(soup.find_all('p', class_="date-time"), soup.find_all('div', class_="text")):
        params1.append(link1.get_text())
        params2.append(link2.get_text())
        
    for i1, i2 in zip(params1, params2):
        #print(i1.strip(), end=' ')
        print(i2.strip())
        
contents_hankyoreh()
    

### 한국경제

In [ ]:
#한국경제와 같은 경우 

import re
import urllib.request
from bs4 import BeautifulSoup

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def contents_hankyung():
    
    #샘플 기사 URL
    list_url = "https://www.hankyung.com/finance/article/2021122646691"
    
    url = urllib.request.Request(list_url, headers={'User-Agent': 'Custom'}) #
    res = urllib.request.urlopen(url).read().decode("utf-8")
    
    soup = BeautifulSoup(res, "html.parser")
    
    content_obj = soup.find_all('div', id="articletxt")
    content = cleanhtml(str(content_obj))
    return content
        

contents_hankyung()
    

### 경향신문

In [ ]:
#샘플 기사 URL
list_url = 'https://www.khan.co.kr/national/national-general/article/202204142115005'

def contents_kyunghyang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('p', class_="content_text")

    content = cleanhtml(str(content_obj))
    return content

contents_kyunghyang(list_url)

### 동아일보

In [ ]:
list_url = 'https://www.donga.com/news/article/all/20220404/112693737/1'

def contents_donga(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_txt")

    content = cleanhtml(str(content_obj))
    return content

contents_donga(list_url)

### 중앙일보

In [ ]:
list_url = 'https://news.joins.com/article/2137057'

def contents_joongang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_body")

    content = cleanhtml(str(content_obj))
    return content

contents_joongang(list_url)

### 매일경제

In [ ]:
import bs4
list_url = 'https://www.mk.co.kr/news/stock/view/2022/04/339384/'

def contents_maeil(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
    soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

    content_obj = soup_alt.find_all('div', class_="art_txt")

    content = clean_html(str(content_obj))
    return content

contents_maeil(list_url)

### 조선일보

In [5]:
!pip install selenium

In [6]:
#조선일보 같은 경우 셀레니움 패키지를 사용

import re
import requests
from bs4 import BeautifulSoup
import urllib.request
import selenium
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(ChromeDriverManager().install())

list_url = 'https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?utm_source=bigkinds&utm_medium=original&utm_campaign=news'

content_obj = []

#Legacy Component
def contents_chosun(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")
    print(soup)
    content_obj = soup.find_all('p')
    print(content_obj)
    
    content = clean_html(str(content_obj))
    
    if soup.select_one('#text'):
        soup.dl.extract()
        news_article = soup.select_one('#article').text
        print(news_article)
    
#    for tag in soup.select('#text'):
#        print(tag.text)
#        write_ws.append([tag.text])
    
#    content = clean_html(str(content_obj))
    return content

contents_chosun(list_url)

<!DOCTYPE html>
<html lang="ko"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/><title>불확실성 커지는 내년 재테크, 홈런보다는 안타 노려라 - 조선일보</title><link href="https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/" rel="canonical"/><meta content="불확실성 커지는 내년 재테크, 홈런보다는 안타 노려라 대한민국 재테크 박람회, 참가자 절반이 2030 오미크론 코로나 변이와 내년 대선, 미국의 테이퍼링양적 완화 축소 등 각종 변수는 내년 주식시장에 어떤 영향을 미칠까. 강남 집값이 연일 신고가를 기록하는 가운데 가격" name="description"/><link href="https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?outputType=amp" rel="amphtml"/><meta content="https://images.chosun.com/resizer/pDZHe1ckoSUa_sc5XmzrnelcW1w=/700x367/smart/cloudfront-ap-northeast-1.images.arcpublishing.com/chosun/U5ZOVHQNPVAPDFCQQKG3PN5P5Y.jpg" property="og:image"/><meta content="article" property="og:type"/><meta content="https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/" property="og

''

In [7]:
#셀레니윰을 통한 조선일보 크롤링 접근

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

import pandas as pd
from selenium.common.exceptions import NoSuchElementException

list_url = 'https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?utm_source=bigkinds&utm_medium=original&utm_campaign=news'

content = []


#Selenium Component
driver = webdriver.Chrome(executable_path='/Users/ohikendoit/chromedriver', options=options)
driver.get(list_url)

driver.implicitly_wait(time_to_wait=5)
print(driver)

#more = driver.find_element_by_id('load-more-stories')

#for _ in range(120):
#    more.click()
#    time.sleep(1)
    
#selector = '#artwrapper > div > figure > div > div > div'
#a = driver.find_element(selector)

try:
    content_section = driver.find_element('//*[@id="fusion-app"]/div[1]/div[2]/div/section/article/section')
    p = content_section.find_elements('p')
    temp = []
    for item in p:
        temp.append(item.text)
    content.append('\n'.join(temp))
except:
    content.append(pd.NA)

driver.close()

/var/folders/s2/hnmqp7lj4t1bmkwjn8qdwk3w0000gn/T/ipykernel_63930/1293437698.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/ohikendoit/chromedriver', options=options)


<selenium.webdriver.chrome.webdriver.WebDriver (session="5e17517ac4c4ccd95084c6b57a7f88c3")>


## 셀레니움을 통한 조선일보 크롤링 구현

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [9]:
driver = webdriver.Chrome(executable_path='/Users/ohikendoit/chromedriver', options=options)
try:
    driver.get("https://www.google.com/")
except Exception as e:
    print(e.message)

/var/folders/s2/hnmqp7lj4t1bmkwjn8qdwk3w0000gn/T/ipykernel_63930/3015450984.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/ohikendoit/chromedriver', options=options)


In [10]:
driver.get("https://www.google.com/")

In [ ]:
start_date = '20200501'
end_date = '20200502'
paper = 1023

num = 1
while True:
    driver.get(f'https://search.naver.com/search.naver?where=news&query=%EC%A7%80%EC%97%AD%ED%99%94%ED%8F%90&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date[0:4]}.{start_date[4:6]}.{start_date[6:]}&de={end_date[0:4]}.{end_date[4:6]}.{end_date[6:]}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={paper}&nso=so%3Ar%2Cp%3Afrom{start_date}to{end_date},a:all&start={num}')
    time.sleep(3)
    
    try:
        ul = driver.find_element(By.CSS_SELECTOR, '#main_pack > section > div > div.group_news > ul')
    except NoSuchElementException:
        print("Not exist paper")
        break
    lis = ul.find_elements(By.CLASS_NAME, 'li') #The previous approach has been depreciated.
    
    check = False
    
    for li in lis:
        tmp = li.get_attribute('id')
        tmp_url = driver.find_element(By.CSS_SELECTOR, f'#{tmp} > div > div > a').get_attribute("href")
        if tmp_url in url_list:
            check = True
            break
        url_list.append(tmp_url)
        
    if check:
        break
        
    num += 10

In [11]:
url_list = ['https://biz.chosun.com/site/data/html_dir/2020/04/23/2020042302274.html?utm_source=bigkinds&utm_medium=original&utm_campaign=biz']

In [19]:
for url in url_list:
    driver.get(url)
    time.sleep(1)
    content = driver.find_element_by_xpath('//*[@id="fusion-app"]').text
    print(content)
    
    print('='*30)
    print('='*30)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x00000001069a12c9 chromedriver + 5120713
1   chromedriver                        0x000000010692fe33 chromedriver + 4656691
2   chromedriver                        0x000000010651f00f chromedriver + 393231
3   chromedriver                        0x0000000106546fc4 chromedriver + 556996
4   chromedriver                        0x00000001065713fc chromedriver + 730108
5   chromedriver                        0x000000010656f195 chromedriver + 721301
6   chromedriver                        0x000000010656ea37 chromedriver + 719415
7   chromedriver                        0x00000001064f7b6a chromedriver + 232298
8   chromedriver                        0x0000000106973b1d chromedriver + 4934429
9   chromedriver                        0x0000000106978295 chromedriver + 4952725
10  chromedriver                        0x000000010697d3cf chromedriver + 4973519
11  chromedriver                     